In [3]:
from dataset_utils import *
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [5]:
import os
os.listdir("/data2/linhang_data/Earthquake_data/data/")

['Southern California', 'Japan', 'California']

In [2]:
area = "Japan"
data_path = "/data2/linhang_data/Earthquake_data/data/"+area+"/"


window_size = 14 * 102
forecast_horizon = 14

lape_dim = 30

far_mask_delta = 30
dtw_delta = 10


gnss_data = pd.read_csv(data_path+"gnss_data.csv", index_col=0, parse_dates=True).map(parse_str_list)
earthquake_data = pd.read_csv(data_path+"earthquake_data.csv", index_col=0, parse_dates=True)
energy_data = pd.read_csv(data_path+"energy_data.csv", index_col=0, parse_dates=True)
station_dict_use = pickle.load(open(data_path+"station_dict_use.pkl", "rb"))

es_geo_matrix = pd.read_csv(data_path+"es_geo_matrix.csv", index_col=0)
es_sem_matrix = pd.read_csv(data_path+"es_sem_matrix.csv", index_col=0)
gnss_geo_matrix = pd.read_csv(data_path+"gnss_geo_matrix.csv", index_col=0)

dataset_japan = EarthquakeGNSSDataset(area=area,earthquake_data=earthquake_data,es_geo_matrix=es_geo_matrix,es_sem_matrix=es_sem_matrix,
                                gnss_geo_matrix=gnss_geo_matrix,gnss_data=gnss_data,far_mask_delta=far_mask_delta,
                                dtw_delta=dtw_delta,lape_dim=lape_dim,
                                window_size=window_size,forecast_horizon=forecast_horizon,earthquake_threshold=4)

In [3]:
area = "California"
data_path = "/data2/linhang_data/Earthquake_data/data/"+area+"/"


window_size = 14 * 102
forecast_horizon = 14

lape_dim = 30

far_mask_delta = 30
dtw_delta = 10


gnss_data = pd.read_csv(data_path+"gnss_data.csv", index_col=0, parse_dates=True).map(parse_str_list)
earthquake_data = pd.read_csv(data_path+"earthquake_data.csv", index_col=0, parse_dates=True)
energy_data = pd.read_csv(data_path+"energy_data.csv", index_col=0, parse_dates=True)
station_dict_use = pickle.load(open(data_path+"station_dict_use.pkl", "rb"))

es_geo_matrix = pd.read_csv(data_path+"es_geo_matrix.csv", index_col=0)
es_sem_matrix = pd.read_csv(data_path+"es_sem_matrix.csv", index_col=0)
gnss_geo_matrix = pd.read_csv(data_path+"gnss_geo_matrix.csv", index_col=0)

dataset_California = EarthquakeGNSSDataset(area=area,earthquake_data=earthquake_data,es_geo_matrix=es_geo_matrix,es_sem_matrix=es_sem_matrix,
                                gnss_geo_matrix=gnss_geo_matrix,gnss_data=gnss_data,far_mask_delta=far_mask_delta,
                                dtw_delta=dtw_delta,lape_dim=lape_dim,
                                window_size=window_size,forecast_horizon=forecast_horizon,earthquake_threshold=4)

In [4]:
for key in dataset_japan[0]:
    print(key,dataset_japan[0][key].shape)
mask = dataset_japan.get_masks()
for key in mask:
    print(key,mask[key].shape)

log_energy_history torch.Size([102, 100, 1])
gnss_data_history torch.Size([1428, 334, 4])
log_energy_future torch.Size([100, 1])
earthquake_data_future_day torch.Size([100])
es_geo_mask torch.Size([100, 100])
es_sem_mask torch.Size([100, 100])
gnss_geo_mask torch.Size([334, 334])
lap_ex torch.Size([100, 30])
lap_gnss torch.Size([334, 30])


In [5]:
for key in dataset_California[0]:
    print(key,dataset_California[0][key].shape)
mask = dataset_California.get_masks()
for key in mask:
    print(key,mask[key].shape)

log_energy_history torch.Size([102, 100, 1])
gnss_data_history torch.Size([1428, 294, 4])
log_energy_future torch.Size([100, 1])
earthquake_data_future_day torch.Size([100])
es_geo_mask torch.Size([100, 100])
es_sem_mask torch.Size([100, 100])
gnss_geo_mask torch.Size([294, 294])
lap_ex torch.Size([100, 30])
lap_gnss torch.Size([294, 30])


In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

class CombinedEarthquakeGNSSDataset(Dataset):
    def __init__(self, region_datasets):
        """
        Class to combine datasets from different regions.
        
        :param region_datasets: Dictionary where keys are region names and values are instances of EarthquakeGNSSDataset.
        """
        self.region_datasets = region_datasets
        self.region_names = list(region_datasets.keys())
        self.lengths = {region: len(ds) for region, ds in region_datasets.items()}
        self.total_length = sum(self.lengths.values())
        self.max_gnss_nodes = max(ds[0]['gnss_data_history'].shape[1] for ds in region_datasets.values())

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        """
        Retrieve an item based on the global index. Maps the global index to a specific region dataset.
        """
        for region in self.region_names:
            if idx < self.lengths[region]:
                data = self.region_datasets[region][idx]
                data['region'] = region  # Include the region name in the data for identification.
                return data
            idx -= self.lengths[region]
        raise IndexError("Index out of range in CombinedEarthquakeGNSSDataset.")

    def collate_fn(self, batch):
        """
        Custom collate function to handle batches with different numbers of GNSS nodes.
        
        :param batch: List of data samples from different regions.
        :return: A dictionary containing combined batched data with region-specific masks.
        """
        # Determine the maximum number of GNSS nodes in this batch.
        max_gnss_nodes_in_batch = max(item['gnss_data_history'].shape[1] for item in batch)

        # Initialize lists to store the padded data and masks.
        log_energy_history = []
        gnss_data_history = []
        log_energy_future = []
        earthquake_data_future_day = []
        es_geo_masks = []
        es_sem_masks = []
        combined_gnss_masks = []
        lap_ex_masks = []
        lap_gnss_masks = []

        # Pad each data sample's GNSS data and masks to match the maximum number of nodes.
        for item in batch:
            n_gnss_nodes = item['gnss_data_history'].shape[1]
            pad_size = max_gnss_nodes_in_batch - n_gnss_nodes

            # Pad the GNSS data along the node dimension (dim=1)
            padded_gnss_data = F.pad(item['gnss_data_history'], (0, 0, 0, pad_size))
            # Append the data.
            log_energy_history.append(item['log_energy_history'])
            gnss_data_history.append(padded_gnss_data)
            log_energy_future.append(item['log_energy_future'])
            earthquake_data_future_day.append(item['earthquake_data_future_day'])

            # Retrieve region-specific masks.
            region_name = item['region']
            region_masks = self.region_datasets[region_name].get_masks()

            # Pad 'lap_gnss' and 'gnss_geo_mask' to match the maximum number of nodes.
            padded_gnss_geo_mask = F.pad(region_masks['gnss_geo_mask'], (0, pad_size,0, pad_size), value=1)

            padded_lap_gnss = F.pad(region_masks['lap_gnss'], (0, 0, 0, pad_size))
            # Combine GNSS padding mask with padded GNSS geo mask.
            gnss_padding_mask = torch.cat([torch.ones(n_gnss_nodes), torch.zeros(pad_size)], dim=0)
            combined_gnss_mask = gnss_padding_mask.unsqueeze(-1) * padded_gnss_geo_mask

            # Store the masks.
            es_geo_masks.append(region_masks['es_geo_mask'])
            es_sem_masks.append(region_masks['es_sem_mask'])
            combined_gnss_masks.append(combined_gnss_mask)
            lap_ex_masks.append(region_masks['lap_ex'])
            lap_gnss_masks.append(padded_lap_gnss)

        # Stack the data to create tensors for the batch.
        log_energy_history = torch.stack(log_energy_history)
        gnss_data_history = torch.stack(gnss_data_history)
        log_energy_future = torch.stack(log_energy_future)
        earthquake_data_future_day = torch.stack(earthquake_data_future_day)

        # Combine the padded masks.
        es_geo_masks = torch.stack(es_geo_masks)
        es_sem_masks = torch.stack(es_sem_masks)
        combined_gnss_masks = torch.stack(combined_gnss_masks)
        lap_ex_masks = torch.stack(lap_ex_masks)
        lap_gnss_masks = torch.stack(lap_gnss_masks)

        # Combine into a dictionary.
        batch_data = {
            'log_energy_history': log_energy_history,
            'gnss_data_history': gnss_data_history,
            'log_energy_future': log_energy_future,
            'earthquake_data_future_day': earthquake_data_future_day,
            'es_geo_mask': es_geo_masks,
            'es_sem_mask': es_sem_masks,
            'combined_gnss_mask': combined_gnss_masks,
            'lap_ex': lap_ex_masks,
            'lap_gnss': lap_gnss_masks
        }

        return batch_data

# Example usage:
# region_datasets = {
#     'region_1': dataset_japan,
#     'region_2': dataset_california,
# }
# combined_dataset = CombinedEarthquakeGNSSDataset(region_datasets)
# dataloader = DataLoader(combined_dataset, batch_size=32, collate_fn=combined_dataset.collate_fn)

In [32]:
region_datasets = {"Japan": dataset_japan, "California": dataset_California}
dataset = CombinedEarthquakeGNSSDataset(region_datasets)

In [33]:
print("Dataset length:", len(dataset),region_datasets["Japan"].__len__(),region_datasets["California"].__len__())

Dataset length: 40 20 20


In [34]:
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=dataset.collate_fn)
for batch in dataloader:
    print("Batch data:")
    for key, value in batch.items():
        print(key, value.shape)
    break

torch.Size([334]) torch.Size([334, 334])
torch.Size([334]) torch.Size([334, 334])
torch.Size([334]) torch.Size([334, 334])
torch.Size([334]) torch.Size([334, 334])
torch.Size([334]) torch.Size([334, 334])
torch.Size([334]) torch.Size([334, 334])
torch.Size([334]) torch.Size([334, 334])
torch.Size([334]) torch.Size([334, 334])
Batch data:
log_energy_history torch.Size([8, 102, 100, 1])
gnss_data_history torch.Size([8, 1428, 334, 4])
log_energy_future torch.Size([8, 100, 1])
earthquake_data_future_day torch.Size([8, 100])
es_geo_mask torch.Size([8, 100, 100])
es_sem_mask torch.Size([8, 100, 100])
combined_gnss_mask torch.Size([8, 334, 334])
lap_ex torch.Size([8, 100, 30])
lap_gnss torch.Size([8, 334, 30])
